In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt

################################################################################
ML_Type = 'Class'
Gene_Type = 'Common'
Feature_Size = 2100  # Feature size {50, 100, 200, 400, 600, 800, 1000}
print(ML_Type)
print(Gene_Type)
#print("Size of the Columns/Features is: %d." %(Feature_Size))
dataDir = 'E:/1st Project/'
dataDir = dataDir + str(ML_Type) + '/' + str(Gene_Type) + '/' + str(Feature_Size) + '/'
outputDir = dataDir
dataType = '';
print(dataDir)
################################################################################
indx = 1
print(indx)
prt1 = 'X_orig_'+str(indx)+'.csv'
prt2 = 'X_ko1_'+str(indx)+'.csv'; prt3 = 'X_ko2_'+str(indx)+'.csv'; prt4 = 'X_ko3_'+str(indx)+'.csv'
prt5 = 'X_ko4_'+str(indx)+'.csv'; prt6 = 'X_ko5_'+str(indx)+'.csv';
prt7 = 'Y_'+str(indx)+'.csv'; prt8 = 'Beta_'+str(indx)+'.csv';
#prt9 = 'Pheno_Data' + '.csv';
output_path1 = dataDir + prt1
output_path2 = dataDir + prt2; output_path3 = dataDir + prt3; output_path4 = dataDir + prt4;
output_path5 = dataDir + prt5; output_path6 = dataDir + prt6;
output_path7 = dataDir + prt7; output_path8 = dataDir + prt8
#output_path9 = dataDir + prt9

X_orig = pd.read_csv(output_path1, header = None).values.astype(np.float32);
X_ko1 = pd.read_csv(output_path2, header = None).values.astype(np.float32);
X_ko2 = pd.read_csv(output_path3, header = None).values.astype(np.float32);
X_ko3 = pd.read_csv(output_path4, header = None).values.astype(np.float32);
X_ko4 = pd.read_csv(output_path5, header = None).values.astype(np.float32);
X_ko5 = pd.read_csv(output_path6, header = None).values.astype(np.float32);
Y = pd.read_csv(output_path7, header = None).values.astype(np.float32);
Beta = pd.read_csv(output_path8, header = None).values.astype(np.float32);
#Pheno = pd.read_csv(output_path9, header = None).values.astype(np.float32);

print("Size of the original feature is: %d x %d." %(X_orig.shape))
print("Size of the knockoff feature is: %d x %d." %(X_ko4.shape))
print("Size of the target is: %d x %d." %(Y.shape))
print("Size of the output weight is: %d x %d." %(Beta.shape))

X_all = np.concatenate((X_orig,X_ko1),axis=1);
Num_knock = 1;
bias = True;
validation_split = 0.1
n = X_all.shape[0];
X_dim = X_all.shape[1];

####################################################
num_epochs = 5;

seed = 999
np.random.seed(seed)

num_row = X_all.shape[0];
X_dim = X_all.shape[1];
x3D_all = np.zeros((num_row, X_orig.shape[1] , Num_knock+1));
x3D_all[:, :, 0] = X_orig;
x3D_all[:, :, 1] = X_ko1; 


def Normalize(data, mean_data =None, std_data =None):
    if not mean_data:
        mean_data = np.mean(data)
    if not std_data:
        std_data = np.std(data)
    norm_data = (data-mean_data)/std_data
    return norm_data, mean_data, std_data

num_row = X_orig.shape[0];
x3D_all = np.zeros((num_row, X_orig.shape[1] , Num_knock+1));
x3D_all[:, :, 0] = X_orig;
x3D_all[:, :, 1] = X_ko1; 
x3D_all.shape

del X_ko1

x3D_all = (x3D_all-np.mean(x3D_all))/np.std(x3D_all);

Kernel_Size = 1
STRIDE = 1
############ Zero-Padding ###############################
print(f'The kernel size or number of neurons per group is: {Kernel_Size}')
Var_dim = x3D_all[:, :, 0].shape[1]
print(f'The total number of dimensions is: {Var_dim}')
res = Var_dim%Kernel_Size
print(f'The residual is: {res}')

Num_group = int(x3D_all.shape[1]/STRIDE); 
Size_group1 = Num_group*Kernel_Size
print(f'The total size of neurons for first locally connected layer is: {Size_group1}')

print(f'number of features/columns after zero padding: {x3D_all.shape[1]}')
#########################################################
pVal = x3D_all.shape[1];
Num_instance = x3D_all.shape[0];
seed = 457
bias = True;


In [ ]:
x3D_all.shape

In [ ]:
# test_net.py
import torch
import tensorflow as tf
Y = torch.from_numpy(Y)
x3D_all2 = torch.from_numpy(x3D_all)
x3D_all2.requires_grad=True
type(x3D_all2)

In [ ]:
# out_channels: The size of the new dimension is called the number of output channels, or number of output feature maps
# In case of an RGB image, the size of the new dimension (aside from the width/height of the image) is 3.
#This "size of the 3rd dimension" is called the number of input channels or number of input feature maps.
#In the above image example, the number of input channels is 3, and we have a 3×3×3 kernel.
#(in_channels=2,out_channels=1,output_size=98,kernel_size=1,stride=1)

import torch.nn as nn
class LocallyConnected1d(nn.Module):
    def __init__(self, in_channels, out_channels, output_size, kernel_size, stride, bias=False):
        super(LocallyConnected1d, self).__init__()
        self.weight = nn.Parameter(
            torch.randn(1, out_channels, in_channels, output_size, kernel_size)
        )
        if bias:
            self.bias = nn.Parameter(
                torch.randn(1, out_channels, output_size)
            )
        else:
            self.register_parameter('bias', None)
        self.kernel_size = kernel_size
        self.stride = stride
        
    def forward(self, x):
        #_, c, h = x.size()
        _,c, h = x.size()
#        print(c)
#        print(h)
        kh = self.kernel_size
        print(kh)
        dh = self.stride
        print(dh)
        #x = x.unfold(2, kh, dh)
        x = x.unfold(1, kh, dh)
        print(x.shape)
        # x = x.contiguous().view(*x.size()[:-2], -1)
        # Sum in in_channel and kernel_size dims
        x = x.unsqueeze(1)
        out = (x * self.weight)
        out = out.sum([2, -1])
        if self.bias is not None:
            out += self.bias
        return out

In [ ]:
# Hyper-parameters 
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.nn import BCELoss, Linear, Module, ReLU, Sigmoid
from torch.nn.init import kaiming_uniform_, xavier_uniform_
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 2048
input_size = 98
input_size2 = 98
hidden_size = 20
learning_rate = 0.001
from torch.nn import functional as F

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.loc1 = LocallyConnected1d(2,1,input_size,1,1)
        xavier_uniform_(self.loc1.weight)
        self.ActLoc1 = torch.nn.functional.elu
        self.fc1 = nn.Linear(input_size2, hidden_size) 
        xavier_uniform_(self.fc1.weight)
        self.Act1 = torch.nn.functional.elu
        self.fc2 = nn.Linear(hidden_size, hidden_size)  
        xavier_uniform_(self.fc2.weight)
        self.Act2 = torch.nn.functional.elu
        self.fc3 = nn.Linear(hidden_size, 1)
        xavier_uniform_(self.fc3.weight)
        self.Act3 = Sigmoid()
    
    def forward(self, x):
        out = self.loc1(x)
        out = self.ActLoc1(out)
        out = self.fc1(out)
        out = self.Act1(out)
        out = self.fc2(out)
        out = self.Act2(out)
        out = self.fc3(out)
        out = self.Act3(out)
        return out
        
    def compute_l1_loss(self, w):
        return torch.abs(w).sum()        
    

Model = NeuralNet(input_size, hidden_size).to(device)

# Loss and optimizer
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss() # initialize loss function
optimizer = torch.optim.Adam(Model.parameters(), lr=learning_rate)  


In [ ]:
# Train the model
grad_bank = {}
avg_counter = 0
num_ins = x3D_all2.shape[0]
num_epochs = 40
p_norm = 1
l1_lambda = 0.00001
Batch_counter = int(np.ceil(num_ins/batch_size))
total_step = Batch_counter
print(Batch_counter)
for epoch in range(num_epochs):   
    for step in range(Batch_counter):
        print("The step is", step+1, "out of", Batch_counter)
        step1 = (step*batch_size)
        if step == (Batch_counter-1):
            step2 = num_ins
        else: 
            step2 = (step+1)*batch_size
        print("The step1 is", step1, "and step2 is", step2, "out of",num_ins)
        # Forward pass
        X_batch = x3D_all2[step1:step2,].clone().requires_grad_(True)
        outputs = Model(X_batch) #X_all2[step1:step2,]
        outputs = torch.squeeze(outputs, 0)
        print("The output size is", outputs.shape)
        loss = criterion(outputs, Y[step1:step2,])  
        #regularity =  torch.norm(model.loc1.weight, p=p_norm)
        #cost = loss + l1_lambda * regularity
        
      # Compute L1 loss component
        l1_weight = 1.0
        l1_parameters = []
        for parameter in Model.parameters():
            l1_parameters.append(parameter.view(-1))
        l1 = l1_weight * Model.compute_l1_loss(torch.cat(l1_parameters))
      
      # Add L1 loss component
        loss += l1        
        
        # Backward and optimize
        optimizer.zero_grad()
        # Compute gradients
        loss.backward()
        
        #for idx, param in enumerate(model.parameters()):
         #   grad_bank[idx] += param.grad.data
          #  avg_counter += 1        
        
        optimizer.step()

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, step+1, total_step, loss.item()))

# Test the model
# evaluate the model
yhat = model(x3D_all2)
yhat = yhat.detach().numpy()
yhat.shape
actual = Y.numpy()
actual = actual.reshape((len(actual), 1))
# round to class values
yhat = yhat.round()
yhat = yhat.reshape((num_ins, 1))
from sklearn.metrics import accuracy_score
acc = accuracy_score(Y, yhat)
acc


In [ ]:
weight = nn.Parameter(torch.randn(1, 1, 2, 98, 1))
weight.shape

In [ ]:
X_batch.shape

In [ ]:
_,c, h = X_batch.size()
c

In [ ]:
kernel_size = stride = 1

kh = kernel_size
print(kh)
dh = stride
print(dh)

In [ ]:
x = X_batch.unfold(1, kh, dh)
print(x.shape)

In [ ]:
x2 = x.unsqueeze(1)
x2.shape

In [ ]:
weight.shape

In [ ]:
out = (x2 * weight)

In [ ]:
out = out.sum([2, -1])

In [ ]:
weight = nn.Parameter(torch.randn(1, 1, 1, 98, 2))

In [ ]:
c, h = x.size()

In [ ]:
import torch
import tensorflow as tf
Y = torch.from_numpy(Y)
X_all2 = torch.from_numpy(X_all)
X_all2.requires_grad=True
type(X_all2)
# Train the model
grad_bank = {}
avg_counter = 0
num_ins = X_all2.shape[0]
num_epochs = 40
p_norm = 1
l1_lambda = 0.00001
Batch_counter = int(np.ceil(num_ins/batch_size))
total_step = Batch_counter
print(Batch_counter)